# MedleyDB dataset
This project uses the [MedleyDB dataset](https://medleydb.weebly.com/) V1 (and V2). To get access to the dataset, you need to ask for permission through Zenodo. \
This notebook is created to explore the MedleyDB dataset. There is an overview of the dataset here: https://medleydb.weebly.com/description.html \
Fortunately, the authors provides a sample of the dataset here: https://zenodo.org/record/1438309#.Y9dniuxBy3I. The sample set contains 2 songs from V1:
1. LizNelson_Rainfall
2. Phoenix_ScotchMorris

## Setups
To correctly setup the medleydb library, the `medleydb/medleydb/data/` folder need to have the structure below:
```
medleydb #sub-module
├── medleydb
│   ├── data
│   │   ├── Annotations
│   │   ├── Audio
│   │   └── Metadata
...
```
Simply rename `V1` or `V2` folder after extracting to `Audio` and put it inside the `data/` folder above. \
For the MedleyDB Sample (2 songs) simply copy-paste `Audio` folder into `data/`.

For accessibility, we will try to use only the 2 songs in the sample set for making example.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import random

import numpy as np
import pandas as pd

from pitch_tracker.utils import files

If import medleydb return `TypeError: load() missing 1 required positional argument: 'Loader'`, you need to downgrade pyyaml:
```
pyyaml==5.4.1
```

In [ ]:
# path to a forked medleydb repo https://github.com/duotien/medleydb
MEDLEYDB_REPO_PATH = f"{os.getcwd()}/../medleydb/"
MEDLEY_DB_DATA_PATH = os.path.join(MEDLEYDB_REPO_PATH, 'medleydb/data/')
print(MEDLEY_DB_DATA_PATH)
os.environ['MEDLEYDB_PATH'] = MEDLEY_DB_DATA_PATH
import medleydb
from pitch_tracker.utils.medleydb_melody import gen_label

# Generate annotations
medleyDB also has tools for customizing the annotations. However you need to have the audio for it to work.

In [ ]:
HOP = 512*5
output_dir = f'../content/gen_label/{HOP}'
AUDIO_PATH = medleydb.AUDIO_PATH

available_tracks = list(files.list_folder_paths_in_dir(AUDIO_PATH))
available_tracks = [files.get_file_name(full_path) for full_path in available_tracks]

mtracks = medleydb.load_multitracks(available_tracks)

for mtrack in mtracks:
    gen_label(mtrack.track_id, output_dir, hop=HOP, overwrite=True, convert_to_midi=True, round_method='round', to_csv=True)
    # gen_label(mtrack.track_id, output_dir, hop=HOP, overwrite=True, convert_to_midi=True, round_method='round', to_csv=False)
    pass

In [ ]:
melody_1_path, melody_2_path, *_ = [os.path.join(output_dir, folder_path) for folder_path in os.listdir(output_dir)]

In [ ]:
melody_2_annotation_paths = []
for root, folders, files in os.walk(melody_2_path):
    melody_2_annotation_paths.extend([os.path.join(root, fpath) for fpath in files])

In [ ]:
sample_annotation_path = random.choice(melody_2_annotation_paths)
print(sample_annotation_path)
df = pd.read_csv(sample_annotation_path, names=['start', 'end', 'midi_value'])
df.head(5)

# Visualise
We recommend using Sonic Visualiser to visualize the annotations with audios.

In [ ]:
# code to visualize using matplotlib
from matplotlib import pyplot as plt
import librosa
from librosa.display import specshow, waveshow
import torchaudio

In [ ]:
def plot_spectrogram(signal: np.ndarray, sample_rate:int, convert_to_mel:bool=False,offset:int=0, n_frame:int=None, f_ratio=1, n_mels:int=108, **stft_params):
    if len(signal.shape) == 1:
        signal = np.expand_dims(signal, axis=0)
    if n_frame is not None:
        signal = signal[:,offset:offset+n_frame]
    
    # Apply FFT & Calculate the Magnitude Spectrum


    if convert_to_mel:
        S = librosa.feature.melspectrogram(y=signal, sr=sample_rate, **stft_params, n_mels=n_mels)
        S_log = librosa.power_to_db(S, ref=np.max)
        y_axis = 'mel'

    else:
        S = librosa.stft(y=signal, **stft_params)
        S_log = librosa.amplitude_to_db(np.abs(S), ref=np.max)
        y_axis = 'linear'

    num_channels, num_frames = signal.shape
    figure, axes = plt.subplots(nrows=num_channels, ncols=1)

    # frequency = np.linspace(0, sample_rate, num_frames)
    # num_frequency_bins = int(len(frequency)*f_ratio)

    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        # axes[c].plot(frequency[:num_frequency_bins], magnitude_spectrum[c,:num_frequency_bins])
        im = specshow(S_log[c], sr=sample_rate, x_axis='time', y_axis=y_axis, ax=axes[c])
        # axes[c].grid(True)
        axes[c].label_outer()
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle("Spectrogram")
    figure.colorbar(im, ax=axes, format="%+2.0f dB")

    plt.show(block=False)



In [ ]:
audio_path = '../medleydb/medleydb/data/Audio/Phoenix_ScotchMorris/Phoenix_ScotchMorris_MIX.wav'
metadata = torchaudio.info(audio_path)
signal, sample_rate = torchaudio.load(audio_path)

song_duration = metadata.num_frames/metadata.sample_rate # in seconds
m,s = divmod(song_duration, 60)
print(metadata)
print(f'Song duration: {song_duration:.2f}s ({m:.0f}m{s:.0f}s)')

In [ ]:
stft_params = {
    'n_fft': 1024,
    'win_length': None,
    'hop_length': 512,
    'window': "hann",
    'center': True,
    'dtype': None,
    'pad_mode': "constant",
}


In [ ]:
plot_spectrogram(signal.numpy(), sample_rate, convert_to_mel=False, offset=0, n_frame=10*sample_rate, **stft_params)